In [21]:
# 1. Load packages
import pandas as pd
import altair as alt

# Disable Altair limit so the full BEA dataset can be passed directly into charts.
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [22]:
# Asked ChatGPT for a color scheme for industries, it suggested the following:
INDUSTRY_COLORS = {
    "Agriculture & Fishing": "#1f77b4",
    "Mining & Oil/Gas": "#ff7f0e",
    "Utilities": "#2ca02c",                                     
    "Construction": "#d62728",
    "Manufacturing": "#9467bd",
    "Wholesale trade": "#8c564b",
    "Retail trade": "#e377c2",
    "Transportation and warehousing": "#7f7f7f",
    "Information": "#bcbd22",
    "Finance and insurance": "#17becf",
    "Real Estate": "#aec7e8",
    "Professional & Technical Services": "#ffbb78",
    "Management Services": "#98df8a",
    "Admin & Waste Mgmt Services": "#ff9896",
    "Educational services": "#c5b0d5",
    "Healthcare": "#c49c94",
    "Arts and Entertainment": "#f7b6d2",
    "Accommodation and food services": "#dbdb8d",
    "Other services": "#9edae5",
    "Government": "#ad494a",
    'All industry total': "#000000"

}


In [23]:

# 1. Load and clean BEA industry dataset
file_path = "../data/industry_data.xlsx"
raw_data = pd.read_excel(file_path, sheet_name="Data")
raw_data.columns = raw_data.columns.astype(str).str.strip()

# Identify year columns (digits only)
year_cols = [col for col in raw_data    .columns if col.isdigit()]
print("Detected year columns:", year_cols)

# 2. Reshape into long format
bea_state_industry_earnings = pd.melt(
    raw_data,
    id_vars=["GeoFips", "States", "LineCode", "Industry"],
    value_vars=year_cols,
    var_name="Year",
    value_name="Total in USD Billions"
)

# Clean fields
bea_state_industry_earnings["States"] = (
    bea_state_industry_earnings["States"]
    .astype(str)
    .str.strip()
)

bea_state_industry_earnings["Industry"] = (
    bea_state_industry_earnings["Industry"]
    .astype(str)
    .str.strip()
)

bea_state_industry_earnings["Year"] = bea_state_industry_earnings["Year"].astype(int)

# Remove national totals row
bea_state_industry_earnings = bea_state_industry_earnings[
    bea_state_industry_earnings["States"] != "United States"
]

# 3. Dropdown options
industry_options = sorted(bea_state_industry_earnings["Industry"].unique())
year_options = sorted(bea_state_industry_earnings["Year"].unique())

industry_dropdown = alt.binding_select(
    options=industry_options,
    name="Select Industry: "
)

year_dropdown = alt.binding_select(
    options=year_options,
    name="Select Year: "
)

industry_select = alt.selection_point(
    fields=["Industry"],
    bind=industry_dropdown,
    value=industry_options[0]
)

year_select = alt.selection_point(
    fields=["Year"],
    bind=year_dropdown,
    value=year_options[0]
)

# 4. Build bar chart
chart = (
    alt.Chart(bea_state_industry_earnings)
    .mark_bar()
    .encode(
        x=alt.X("States:N", sort="-y", title="States"),
        y=alt.Y("Total in USD Billions:Q", title="Earnings (USD Billions)"),
        color=alt.Color(
            "Industry:N",
            scale=alt.Scale(
                domain=list(INDUSTRY_COLORS.keys()),
                range=list(INDUSTRY_COLORS.values())
            ),
            legend=None
        ),
        tooltip=[
            "States",
            "Industry",
            "Year",
            "Total in USD Billions"
        ]
    )
    .add_params(industry_select, year_select)
    .transform_filter(industry_select)
    .transform_filter(year_select)
    .properties(
        width=900,
        height=320,
        title="State-wise Earnings Comparison by Industry and Year"
    )
)

# 5. Display and save

chart
chart.save("../charts/chart1.html")  # I asked ChatGPT how to embed my charts in CSS and it suggested
                                    # Save chart as HTML file and embed using <iframe>


Detected year columns: ['2019', '2020', '2021', '2022', '2023', '2024']


In [24]:
# Load and clean BEA industry data
data_path = "../data/industry_data.xlsx"

bea_raw = pd.read_excel(data_path)
bea_raw.columns = bea_raw.columns.astype(str).str.strip()

# Identify year columns
year_cols = [col for col in bea_raw.columns if col.isdigit()]

# Reshape into long format
bea_state_industry = pd.melt(
    bea_raw,
    id_vars=["GeoFips", "States", "LineCode", "Industry"],
    value_vars=year_cols,
    var_name="Year",
    value_name="Total in USD Billions"
)

# Clean string fields
bea_state_industry["States"] = bea_state_industry["States"].astype(str).str.strip()
bea_state_industry["Industry"] = bea_state_industry["Industry"].astype(str).str.strip()

# Convert Year into int
bea_state_industry["Year"] = bea_state_industry["Year"].astype(int)

# Remove BEA "All industry total" row
bea_state_industry = bea_state_industry[
    bea_state_industry["Industry"] != "All industry total"
]

# Wrap industry labels for readability
bea_state_industry["Industry"] = bea_state_industry["Industry"].str.replace(", ", ",\n")

# Dropdown selectors
state_options = sorted(bea_state_industry["States"].unique())
year_options = sorted(bea_state_industry["Year"].unique())

state_dropdown = alt.binding_select(options=state_options, name="Select State: ")
year_dropdown = alt.binding_select(options=year_options, name="Select Year: ")

state_select = alt.selection_point(fields=["States"], bind=state_dropdown, value=state_options[0])
year_select = alt.selection_point(fields=["Year"], bind=year_dropdown, value=year_options[0])

# Build the chart
chart = (
    alt.Chart(bea_state_industry)
    .mark_bar()
    .encode(
        y=alt.Y(
            "Industry:N",
            sort="-x",
            title="Industry",
            axis=alt.Axis(labelLimit=500, labelPadding=2)
        ),
        x=alt.X("Total in USD Billions:Q", title="Earnings (Billions USD)"),
        color=alt.Color(
            "Industry:N",
            scale=alt.Scale(
                domain=list(INDUSTRY_COLORS.keys()),
                range=list(INDUSTRY_COLORS.values())
            ),
            legend=None
        ),
        tooltip=["States", "Industry", "Year", "Total in USD Billions"]
    )
    .add_params(state_select, year_select)
    .transform_filter(state_select)
    .transform_filter(year_select)
    .properties(
        width=900,
        height=450,
        autosize="fit",
        title="Industry Earnings for Selected State and Year"
    )
    .configure_view(
        stroke=None,
        continuousWidth=800
    )
)

# Display and save
chart
chart.save("../charts/chart2.html")
print("Chart saved at ../charts/chart2.html")


Chart saved at ../charts/chart2.html


In [25]:
# Load file trend file 
data_path = "../data/trend.xlsx"
growth_df = pd.read_excel(data_path)

# Clean column names
growth_df.columns = growth_df.columns.astype(str).str.strip()

# Growth period columns already in dataset
growth_periods = ["2019-20", "2020-21", "2021-22", "2022-23", "2023-24"]

# Reshape into long format
trend_long = pd.melt(
    growth_df,
    id_vars=["GeoFips", "States", "LineCode", "Industry"],
    value_vars=growth_periods,
    var_name="Period",
    value_name="GrowthRate"
)

# Clean string fields
trend_long["States"] = trend_long["States"].astype(str).str.strip()
trend_long["Industry"] = trend_long["Industry"].astype(str).str.strip()


# Build dropdown options
state_options = sorted(trend_long["States"].unique())
industry_options = sorted(trend_long["Industry"].unique())

# State dropdown (single-select)
state_dropdown = alt.binding_select(
    options=state_options,
    name="Select State: "
)

state_select = alt.selection_point(
    fields=["States"],
    bind=state_dropdown,
    value=state_options[0]
)

# Industry multi-select (legend click)
industry_select = alt.selection_point(
    fields=["Industry"],
    bind="legend"
)

# Build chart
chart = (
    alt.Chart(trend_long)
    .mark_line(point=True, strokeWidth=3)
    .encode(
        x=alt.X(
            "Period:N",
            sort=growth_periods,
            title="Growth Period (Year-over-Year)"
        ),
        y=alt.Y(
            "GrowthRate:Q",
            title="Growth Rate",
            axis=alt.Axis(format="%", labelOverlap=True)
        ),
        color=alt.Color(
            "Industry:N",
            scale=alt.Scale(
                domain=list(INDUSTRY_COLORS.keys()),
                range=list(INDUSTRY_COLORS.values())
            ),
            legend=alt.Legend(
            title=["Click to (de)select industries", "Shift-click for multi-select"]  # I asked ChatGPT how to have these two in different lines
                                                                                      # it suggested using a list for the title
            )
        ),
        tooltip=[
            "States",
            "Industry",
            "Period",
            alt.Tooltip("GrowthRate:Q", format=".1%")
        ]
    )
    .add_params(state_select, industry_select)
    .transform_filter(state_select)
    .transform_filter(industry_select)
    .properties(
        width=750,
        height=420,
        title="State-by-Industry Growth Trends (2019–2024)"
    )
)

chart
chart.save("../charts/chart3.html")           
print("Chart saved at ../charts/chart3.html")


Chart saved at ../charts/chart3.html
